In [2]:
import requests
import pandas as pd

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=tesco&apikey=demo'
r = requests.get(url)
data = r.json()

print(data)


{'bestMatches': [{'1. symbol': 'TSCO.LON', '2. name': 'Tesco PLC', '3. type': 'Equity', '4. region': 'United Kingdom', '5. marketOpen': '08:00', '6. marketClose': '16:30', '7. timezone': 'UTC+01', '8. currency': 'GBX', '9. matchScore': '0.7273'}, {'1. symbol': 'TSCDF', '2. name': 'Tesco plc', '3. type': 'Equity', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.7143'}, {'1. symbol': 'TSCDY', '2. name': 'Tesco plc', '3. type': 'Equity', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.7143'}, {'1. symbol': 'TCO2.FRK', '2. name': 'TESCO PLC ADR/1 LS-05', '3. type': 'Equity', '4. region': 'Frankfurt', '5. marketOpen': '08:00', '6. marketClose': '20:00', '7. timezone': 'UTC+02', '8. currency': 'EUR', '9. matchScore': '0.5455'}, {'1. symbol': 'TCO0.FRK', '2. name': 'TESCO PLC LS-0633333', '3.

In [14]:
data_df = pd.DataFrame(data)
data_df
data["bestMatches"][0]['1. symbol']

#compare portfolios 
#

'TSCO.LON'